This notebook is for feature generation of training data. This needs all the five  raw csv's of training data given from the KDD. We put the data generated in each cell(each feature extraction), into a new csv. Finally, all generated csv's are merged to form a final csv.

For the purpose of testing the model, we have **attached the Final generated csv** which can be used to train and test the model. Generation of these individual features takes lot of time.

In [0]:
import numpy as np
import pandas as pd

In [6]:
import os
from getpass import getpass
import urllib

cmd_string = 'git clone https://github.com/srivathsa-rao/MOOC-Dropout-Prediction.git MLProject'

os.system(cmd_string)

0

In [0]:
import os
os.chdir('/content/MLProject/submissions/originalData')

In [0]:
!tar -xf train.tar.gz

In [0]:
os.chdir('/content/MLProject/submissions/originalData/train')

In [0]:
!mkdir generated

In [0]:
!mkdir merged

In [0]:
!pwd

/content/MLProject/submissions/originalData/train


In [0]:
#training data

object_data = pd.read_csv('object.csv')
dates_data = pd.read_csv('date.csv')

log_data = pd.read_csv('log_train.csv')
enrollment_data = pd.read_csv('enrollment_train.csv')

truth_data = pd.read_csv('truth_train.csv', names=['enrollment_id', 'label'])


In [0]:

print("Columns in object data is :{}".format(object_data.columns))
print("Columns in date data is :{}".format(dates_data.columns))

print("Columns in log data is :{}".format(log_data.columns))
print("Columns in enrollment data is :{}".format(enrollment_data.columns))

print("Columns in truth data is :{}".format(truth_data.columns))


**Feature Engineering**

We have extraced the number of course the user has enrolled in from the enrollment data. This has been extracted with the intention that the number of courses of an user will have impact on whether the user completes or drops a particular course.

The results are stored as part of - 'number_of_course.csv'

In [0]:
#gives number of course taken by each user.
# no need to handle any data imputations.
username = []
number_of_course = []

for i in range(enrollment_data.shape[0]):
    if enrollment_data['username'][i] not in username:
        username.append(enrollment_data['username'][i])
        number_of_course.append(enrollment_data[enrollment_data['username'] == enrollment_data['username'][i]].shape[0])

user_course = pd.DataFrame({'username':username,'number_of_course':number_of_course})

parallel_courses = user_course.merge(enrollment_data, left_on='username', right_on='username', how='inner')[['enrollment_id', 'number_of_course']]

parallel_courses.to_csv('generated/number_of_course.csv', index=False)

We have extracted parallel enrollments of every user. If a user is enrolled in more courses simulataneously, the probability that he completes all his courses might be low and vice-versa. So, this might have impact on the decision of whether an user will drop or complete the course.

Extracted feature is stored in - parallel_enrollments.csv

In [0]:
#Code to fetch parallel enrollments...
pd.options.mode.chained_assignment = None

username = []
number_of_course = []

def interval_overlaps(a, b):
  return min(a['to'], b['to']) - max(a['from'], b['from']) > np.timedelta64(0)


def count_overlaps(df1):
  #print(df1)
  return pd.Series([df1.apply(lambda x: interval_overlaps(x, df1.iloc[i]), axis=1).sum() - 1 for i in range(len(df1))], df1.index)

def get_parallel_enrollment_cnt(course_id_data_frame):
  if len(course_id_data_frame) < 2:
    return 0
  df = dates_data.loc[dates_data['course_id'].isin(course_id_data_frame)]
  df['from'] = pd.to_datetime(df['from']) 
  df['to'] = pd.to_datetime(df['to'])
  overlap_list = df.groupby([True]*len(df)).apply(count_overlaps).values[0]
  if sum(overlap_list) == 0:
    return 0
  else:
    return sum(overlap_list)//np.count_nonzero(np.array(overlap_list))

for i in range(enrollment_data.shape[0]):
    if enrollment_data['username'][i] not in username:
        username.append(enrollment_data['username'][i])
        user_df = enrollment_data.loc[enrollment_data['username'] == username[-1]]
        number_of_course.append(get_parallel_enrollment_cnt(user_df['course_id']))
    

user_course = pd.DataFrame({'username':username,'parallel_enrollments':number_of_course})

parallel_courses = user_course.merge(enrollment_data, left_on='username', right_on='username', how='inner')[['enrollment_id', 'parallel_enrollments']]

parallel_courses.to_csv('generated/parallel_enrollments.csv', index=False)


We have extracted number of users who have enrolled in a particular course. If a course is interesting or popular, there might be more enrollments and this would result in more completion. On the contrary, if the course is challenging, it might have less enrollments and more drop outs among the enrolled students.

Th extracted data is stored in - number_of_users_per_course.csv

In [0]:
# gives number of users taken the same course. Merge this with the enrollment.
# no need to handle any data imputations.
course_id = []
number_of_users = []

for course in enrollment_data['course_id'].unique():
  course_id.append(course)
  number_of_users.append(len(enrollment_data[enrollment_data['course_id'] == course]))

number_of_enrollments_per_course = pd.DataFrame({'course_id':course_id,'number_of_users_in_course':number_of_users})

merge_enrollments_with_course = enrollment_data.merge(number_of_enrollments_per_course, left_on='course_id', right_on='course_id', how='inner')[['enrollment_id', 'number_of_users_in_course']]
merge_enrollments_with_course.to_csv('generated/number_of_users_per_course.csv', index=False)

The number of modules in the enrolled course which the user has accessed is deduced from the enrollment and log_train csv. This might have an impact on whether a student drops the course or completes course. Higher the number of modules accessed per course might have positive impact toward the completion of the course.

The extracted data is stored in - number_of_module.csv

In [0]:
# Number of module accessed
# no need to handle any data imputations.
enrollment_id = []
number_of_module = []

for enrollment in log_data['enrollment_id'].unique():
  enr = log_data[log_data['enrollment_id'] == enrollment]
  number_of_objects_enr =  len(enr['object'].unique())
  enrollment_id.append(enrollment)
  number_of_module.append(number_of_objects_enr)

number_of_objects = pd.DataFrame({'enrollment_id':enrollment_id,'number_of_module':number_of_module})
number_of_objects.to_csv('generated/number_of_module.csv', index=False)

This feature is an extension of the feature extracted in number_of_module.csv. This generates the percentage of module accessed by the user in a course.
This might show the level of interest of the student toward the course.

Extracted feature is stored in percentage_of_module.csv

In [0]:
# Percentage of module accessed
# no need to handle any data imputations.
enrollment_id = []
percentage_of_module = []

modules_per_course = {}

for course in object_data['course_id'].unique():
  course_data_in_object_data = object_data[object_data['course_id'] == course]
  course_data_in_object_data = course_data_in_object_data.dropna(subset=['children'])
  number_of_module_in_course = len(course_data_in_object_data['module_id'].unique())

  modules_per_course[course] = number_of_module_in_course


for enrollment in log_data['enrollment_id'].unique():
  enr = log_data[log_data['enrollment_id'] == enrollment]
  number_of_objects_enr =  len(enr['object'].unique())
  number_of_modules_in_course = modules_per_course[enrollment_data[enrollment_data['enrollment_id'] == enrollment].course_id.values[0]]
  enrollment_id.append(enrollment)
  percentage_of_module.append(number_of_objects_enr/number_of_modules_in_course)

percentage_of_objects = pd.DataFrame({'enrollment_id':enrollment_id,'percentage_of_module':percentage_of_module})
percentage_of_objects.to_csv('generated/percentage_of_module.csv', index=False)

We have extracted time spent on each browser or server event by the user in the course. This helps us to analyse the amount of time a user spends in accessing each course.


There are lot of things student can perform on a course like watcihng course video, doing an assignment etc. We have extracted the time spent by the user (browser) and the server on each kind of event on modules of the courses.
This is extracted based on the assumption that consecutive events timestamp can be used to generate the time spent on the 1st event.

Extracted data is stored in - time_data.csv

In [0]:
# time duration spent on each event by the user in the course!

import time
from datetime import datetime
from dateutil.parser import parse


event_data_frame = pd.DataFrame(np.zeros((enrollment_data.shape[0],15)), columns = ['enrollment_id','time_access_browser','time_access_server','time_navigate_browser','time_navigate_server','time_problem_browser','time_problem_server','time_page_close_server','time_page_close_browser','time_video_server','time_video_browser','time_discussion_server','time_discussion_browser','time_wiki_server','time_wiki_browser'])
event_data_frame["enrollment_id"] = enrollment_data["enrollment_id"]

enrollment_index = 0
last_seen_enrollment = log_data["enrollment_id"][0]
	
data_frame = event_data_frame.to_dict()

for i in range(log_data.shape[0] -1):
  enrollment_id = log_data["enrollment_id"][i]
  
  if enrollment_id != last_seen_enrollment:
    enrollment_index += 1
    last_seen_enrollment = enrollment_id
  
  event = log_data["event"][i]
  source = log_data["source"][i]

  if log_data["enrollment_id"][i+1] == log_data["enrollment_id"][i] :
    time_spent = (parse(log_data["time"][i+1]) - parse(log_data["time"][i])).total_seconds()
    if  time_spent < 7200:
      time_span = time_spent
    else:
      time_span = 0

  event_type = "time_"+event+"_"+source
  data_frame[event_type][enrollment_index] += time_span

event_data_frame = pd.DataFrame.from_dict(data_frame)
event_data_frame.to_csv('generated/time_data_old.csv', index=False)

In [0]:
time_data = pd.read_csv('generated/time_data_old.csv')
#time_data.columns

# del time_data['time_navigate_browser']
# del time_data['time_discussion_browser']
# del time_data['time_page_close_server']
# del time_data['time_wiki_browser']
# del time_data['time_video_server']

time_data.to_csv('generated/time_data.csv', index=False)


If a user regularly spends more time on a course, it helps us to infer that the user will complete the course and vice-versa. So, we have also extracted the number of logs which are less than 60 seconds in any event. 


This is in extension to the time related feature generated above. This shows the count of time bursts on the events which are less than one minute. Students wouldn't have done anything significant in a minute on a module. More number of such events might have an negative impact on the completion of the course.

Extracted data is stored in the - time_data_60.csv

In [0]:
# number of logs less than 60 seconds on any event

event_data_frame = pd.DataFrame(np.zeros((enrollment_data.shape[0],15)), columns = ['enrollment_id','count_access_browser','count_access_server','count_navigate_browser','count_navigate_server','count_problem_browser','count_problem_server','count_page_close_server','count_page_close_browser','count_video_server','count_video_browser','count_discussion_server','count_discussion_browser','count_wiki_server','count_wiki_browser'])
event_data_frame["enrollment_id"] = enrollment_data["enrollment_id"]

enrollment_index = 0
last_seen_enrollment = log_data["enrollment_id"][0]
	
data_frame = event_data_frame.to_dict()

for i in range(log_data.shape[0] -1):
  enrollment_id = log_data["enrollment_id"][i]
  
  if enrollment_id != last_seen_enrollment:
    enrollment_index += 1
    last_seen_enrollment = enrollment_id
  
  event = log_data["event"][i]
  source = log_data["source"][i]

  if log_data["enrollment_id"][i+1] == log_data["enrollment_id"][i] :
    time_spent = (parse(log_data["time"][i+1]) - parse(log_data["time"][i])).total_seconds()
    if time_spent < 60:
      event_type = "count_"+event+"_"+source
      data_frame[event_type][enrollment_index] += 1

event_data_frame = pd.DataFrame.from_dict(data_frame)
event_data_frame.to_csv('generated/time_data_60_old.csv', index=False)

In [0]:
event_data_frame = pd.read_csv('generated/time_data_60_old.csv')
event_data_frame.columns
# del event_data_frame['count_navigate_browser']
# del event_data_frame['count_discussion_browser']
# del event_data_frame['count_page_close_server']
# del event_data_frame['count_wiki_browser']
# del event_data_frame['count_video_server']

event_data_frame.to_csv('generated/time_data_60.csv', index=False)

The number of days every student spends on the modules of the enrolled course is an important criteria to predict the completion of the coursework. If a student is least interested, he might spend less number of days on the course and vice-versa.
We have extracted this data from the log_data and enrollment.

Extracted data is stored in - active_days.csv

In [0]:
#Fetch the number of active days for each enrollment
import time
from datetime import datetime
from dateutil.parser import parse

enrollments = []
active_days = []

for enrollment in log_data['enrollment_id'].unique():
  number_of_active_days = pd.to_datetime(log_data[log_data['enrollment_id'] == enrollment]['time']).dt.date.unique().shape[0]
  print("Enrollment ID :{}".format(enrollment))
  enrollments.append(enrollment)
  active_days.append(number_of_active_days)


number_of_active_day = pd.DataFrame({'enrollment_id':enrollments,'active_days':active_days})
number_of_active_day.to_csv('generated/active_days.csv', index=False)
# merge_enrollments_with_course.to_csv('train/completion_rate_for_courses.csv', index='False')

#local_enrollment["time"].map(lambda t: t.date()).unique()
#pd.to_datetime(local_enrollment['time']).dt.date.unique().shape[0]
#number_of_active_day


Access of modules by students on the enrolled courses in every week of course can help in knowing the access pattern of the students. High number of access of the modules in all weeks would result in low drop rate for the course. 

The extracted data is stored in - wk_cnts.csv

In [0]:
#Code to fetch activities in weeks....

from datetime import timedelta

dates_data['from'] = pd.to_datetime(dates_data['from'])
dates_data['to'] = pd.to_datetime(dates_data['to'])
log_data['time'] = pd.to_datetime(log_data['time'])

enrollment_ids = []
week_activity_count = [[],[],[],[], []]

def append_wk_counts(start_from, time_values):
  start_date = start_from
  end_date = start_date + pd.Timedelta(timedelta(days = 7))
  for i in range(5):
    week_activity_count[i].append(np.sum((start_date <= time_values) & (end_date > time_values), axis =0 ))
    start_date = end_date
    end_date = start_date + pd.Timedelta(timedelta(days = 7))
  

for i in range(enrollment_data.shape[0]):
    enrollment_ids.append(enrollment_data['enrollment_id'][i])
    start_end = dates_data.loc[dates_data['course_id'] == enrollment_data['course_id'][i]][['from','to']]
    time_values = log_data.loc[log_data['enrollment_id'] == enrollment_data['enrollment_id'][i]]['time']
    append_wk_counts(start_end['from'].values, time_values.values)


wk_cnts= pd.DataFrame({'enrollment_id':enrollment_ids,'week1_activity_count':week_activity_count[0], 'week2_activity_count':week_activity_count[1], 'week3_activity_count':week_activity_count[2], 'week4_activity_count':week_activity_count[3], 'week5_activity_count':week_activity_count[4]})

wk_cnts_csv = wk_cnts.merge(enrollment_data, left_on='enrollment_id', right_on='enrollment_id', how='inner')[['enrollment_id', 'week1_activity_count', 'week2_activity_count', 'week3_activity_count','week4_activity_count', 'week5_activity_count']]

wk_cnts_csv.to_csv('generated/wk_cnts.csv', index=False)

The number of 30 minute user sessions doing different activities on the course modules. Large number of access means, high interest and may result in completion of course.

The extracted data is stored in - wk_session_cnts.csv

In [0]:
#get sessions count..
#activities 

#Code to fetch activities in weeks....

from datetime import timedelta

dates_data['from'] = pd.to_datetime(dates_data['from'])
dates_data['to'] = pd.to_datetime(dates_data['to'])
log_data['time'] = pd.to_datetime(log_data['time'])

enrollment_ids = []
week_session_count = [[],[],[],[], []]
daily_activity_count = [[] for i in range(30)]

def give_number_of_sessions_in_week(start_from, time_values, end_time):
  
  # print("start time of event in week", start_from)
  # print("End time of event in week", end_time)

  start_date = start_from
  end_date = start_date + pd.Timedelta(timedelta(minutes= 30))

  # print("end date 2", end_date)
  session_count = 0
  # print(end_time)
  while end_date < end_time:
    condition = (pd.to_datetime(start_date) <= time_values) & (pd.to_datetime(end_date) > time_values)
    events = np.extract(condition, time_values)

    number_of_events = len(events)
    #print(number_of_events)

    if number_of_events >= 1:
      session_count += 1
    
    start_date = end_date
    end_date = start_date + pd.Timedelta(timedelta(minutes= 30))
  return session_count
    

def append_wk_counts(start_from, time_values):

  #print("start_from 1", start_from)
  start_date = start_from
  end_date = start_date + pd.Timedelta(timedelta(days = 7))

  for i in range(5):

    number_of_session_in_week = 0

    condition = (start_date <= time_values) & (end_date > time_values)

    events = np.extract(condition, time_values)
    #print(len(events))
    if len(events) > 0:
      number_of_session_in_week = give_number_of_sessions_in_week(np.array(events[0]), pd.to_datetime(events), end_date[0])
    else:
       number_of_session_in_week = 0
    #number_of_session_in_week = len(events)
    week_session_count[i].append(number_of_session_in_week)
    start_date = end_date
    end_date = start_date + pd.Timedelta(timedelta(days = 7))
    #print(events)
  

for i in range(enrollment_data.shape[0]):
    enrollment_ids.append(enrollment_data['enrollment_id'][i])
    start_end = dates_data.loc[dates_data['course_id'] == enrollment_data['course_id'][i]][['from','to']]
    time_values = log_data.loc[log_data['enrollment_id'] == enrollment_data['enrollment_id'][i]]['time']
    append_wk_counts(start_end['from'].values, time_values.values)
    print("enrollment id:", i)


wk_cnts= pd.DataFrame({'enrollment_id':enrollment_ids,'week1_session_count':week_session_count[0], 'week2_session_count':week_session_count[1], 'week3_session_count':week_session_count[2], 'week4_session_count':week_session_count[3], 'week5_session_count':week_session_count[4]})

wk_cnts_csv = wk_cnts.merge(enrollment_data, left_on='enrollment_id', right_on='enrollment_id', how='inner')[['enrollment_id', 'week1_session_count', 'week2_session_count', 'week3_session_count','week4_session_count', 'week5_session_count']]
wk_cnts_csv.to_csv('generated/wk_session_cnts.csv', index=False)




Activity count of user on a daily basis on the modules of the course. This helps to know the active days of the user for the duration of 30 days i.e the course duration.

Extracted data is stored in - generated/daily_cnts.csv

In [0]:
#code for generating days in activity

from datetime import timedelta

dates_data['from'] = pd.to_datetime(dates_data['from'])
dates_data['to'] = pd.to_datetime(dates_data['to'])
log_data['time'] = pd.to_datetime(log_data['time'])
enrollment_ids = []
daily_activity_count = [[] for i in range(30)]

def append_wk_counts(start_from, time_values):
  start_date = start_from
  end_date = start_date + pd.Timedelta(timedelta(days = 1))
  for i in range(30):
    daily_activity_count[i].append(np.sum((start_date <= time_values) & (end_date > time_values), axis =0 ))
    start_date = end_date
    end_date = start_date + pd.Timedelta(timedelta(days = 1))
  

for i in range(enrollment_data.shape[0]):
    enrollment_ids.append(enrollment_data['enrollment_id'][i])
    start_end = dates_data.loc[dates_data['course_id'] == enrollment_data['course_id'][i]][['from','to']]
    time_values = log_data.loc[log_data['enrollment_id'] == enrollment_data['enrollment_id'][i]]['time']
    append_wk_counts(start_end['from'].values, time_values.values)


dict_to_csv = {'day{0}_activity_count'.format(i+1):activity for i, activity in enumerate(daily_activity_count)}
dict_to_csv['enrollment_id'] = enrollment_ids
daily_cnts= pd.DataFrame(dict_to_csv)

daily_cnts_csv = daily_cnts.merge(enrollment_data, left_on='enrollment_id', right_on='enrollment_id', how='inner')[dict_to_csv.keys()]

daily_cnts_csv.to_csv('generated/daily_cnts.csv', index=False)

Extracted the count of the access of different modules in a course by the students. Higher the access of modules in different categories might result in high completion rate.

Extracted data is stored in - extracted_category.csv

In [0]:
# fetch category data for each log..

log_object_merge = log_data.merge(object_data, left_on='object', right_on='module_id', how='inner')
df_category = pd.DataFrame(np.zeros((enrollment_data.shape[0],16)), columns=['enrollment_id','about','chapter','course','course_info','html','outlink','problem','sequential','static_tab','vertical','video','combinedopenended','peergrading','discussion','dictation'])
df_category["enrollment_id"] = enrollment_data["enrollment_id"]


last_enrollment = df_category['enrollment_id'][0]
df_category_index = 0

df_category = df_category.to_dict()

sorted_log_object_merge = log_object_merge.sort_values(by=['enrollment_id'],ignore_index=True)

for i in range(sorted_log_object_merge.shape[0] -1 ):
  enrollment_id = sorted_log_object_merge['enrollment_id'][i]

  if enrollment_id != last_enrollment:
    df_category_index += 1
    last_enrollment = enrollment_id

  category_ = sorted_log_object_merge['category'][i]
  df_category[category_][df_category_index] += 1


df = pd.DataFrame.from_dict(df_category)
df.to_csv('generated/extracted_category.csv', index=False)



The data of each of the 39 courses which students can enroll. This will help us know the course which has more enrollments, which might have impact on the number of students completing the course.

Extracted data is stored in - encoded_course.csv

In [0]:
# one hot encoding courses.
encoded_courses= pd.get_dummies(enrollment_data['course_id'], prefix = 'course')
encoded_enrollment = pd.concat([enrollment_data[['enrollment_id']], encoded_courses], axis=1)

encoded_enrollment.to_csv('generated/encoded_course.csv', index=False)

Based on the prior data, we are trying to fetch the percentage of completion of any given course. This would help us determine the likelihood of completion of the course for the future as well.

Extracted data is stored in - completion_rate_for_courses.csv

In [0]:
#do not run for test data - as this is extracted in a different way for test

# Find the success rate of each course 
# no need to handle any data imputations.
enrollment_label_merge = enrollment_data.merge(truth_data, left_on='enrollment_id', right_on='enrollment_id', how='inner')

courses = []
completions = []

for course in enrollment_label_merge['course_id'].unique():
  course_enrollment_data = enrollment_label_merge[enrollment_label_merge['course_id'] == course]
  total = course_enrollment_data.shape[0]
  completion = len(course_enrollment_data[course_enrollment_data['label'] == 0]) # 0 is success
  completion_rate = (completion/total) * 100

  courses.append(course)
  completions.append(completion_rate)

course_completion_rate = pd.DataFrame({'course_id':courses,'overall_course_completion_rate':completions})
merge_enrollments_with_completions = enrollment_data.merge(course_completion_rate, left_on='course_id', right_on='course_id', how='inner')[['enrollment_id', 'overall_course_completion_rate']]
merge_enrollments_with_completions.to_csv('generated/completion_rate_for_courses.csv', index=False)

Based on the prior data, we extract the course completion rate of each users of their courses. This would help us understand the trend of each user course completion and drop outs.

Extracted data is strored in - individual_completion_rate.csv

In [0]:
#get success rate for individual enrollments

# do not run for test data - as this is extracted in a different way for test

number_of_users = len(enrollment_data['username'].unique())
enrollment_truth_merge = enrollment_data.merge(truth_data, left_on='enrollment_id', right_on='enrollment_id', how='inner')

df_total_usernames = pd.DataFrame(np.zeros((number_of_users,4)), columns=['username','total_enrollments','drop_rate', 'completion_rate'])
count = 0
for user in enrollment_truth_merge['username'].unique():
  user_pd = enrollment_truth_merge[enrollment_truth_merge['username'] == user]
  print(user_pd['username'].values[0])
  df_total_usernames.loc[count, 'username'] = user_pd['username'].values[0]
  df_total_usernames.loc[count, 'total_enrollments'] = user_pd.shape[0]
  df_total_usernames.loc[count, 'drop_rate'] = (user_pd[user_pd['label'] == 1].shape[0]) / user_pd.shape[0]
  df_total_usernames.loc[count, 'completion_rate'] = (user_pd[user_pd['label'] == 0].shape[0]) / user_pd.shape[0]
  count += 1

completion_rate = df_total_usernames.merge(enrollment_data, left_on='username', right_on='username', how='inner')[['enrollment_id', 'drop_rate', 'completion_rate']]

completion_rate.to_csv('generated/individual_completion_rate.csv', index=False)

Finally, all generated csv's are merged to produce one final csv. This final csv is used for training and cross-validation to build the model.




In [0]:
#Merge columns

active_days = pd.read_csv('generated/active_days.csv')
completion_rate_for_courses = pd.read_csv('generated/completion_rate_for_courses.csv')

merge1 = active_days.merge(completion_rate_for_courses, left_on='enrollment_id', right_on='enrollment_id', how='inner')


extracted_category = pd.read_csv('generated/extracted_category.csv')
merge2 = merge1.merge(extracted_category, left_on='enrollment_id', right_on='enrollment_id', how='inner')


individual_completion_rate = pd.read_csv('generated/individual_completion_rate.csv')
merge3 = merge2.merge(individual_completion_rate, left_on='enrollment_id', right_on='enrollment_id', how='inner')


number_of_course = pd.read_csv('generated/number_of_course.csv')
merge4 = merge3.merge(number_of_course, left_on='enrollment_id', right_on='enrollment_id', how='inner')


# #FIX this to percentage of modules
number_of_module = pd.read_csv('generated/number_of_module.csv')
merge5 = merge4.merge(number_of_module, left_on='enrollment_id', right_on='enrollment_id', how='inner')

number_of_users_per_course= pd.read_csv('generated/number_of_users_per_course.csv')
merge6 = merge5.merge(number_of_users_per_course, left_on='enrollment_id', right_on='enrollment_id', how='inner')

time_data = pd.read_csv('generated/time_data.csv')
merge7 = merge6.merge(time_data, left_on='enrollment_id', right_on='enrollment_id', how='inner')

time_data_less_than_60 = pd.read_csv('generated/time_data_60.csv')
merge8 = merge7.merge(time_data_less_than_60, left_on='enrollment_id', right_on='enrollment_id', how='inner')

parallel_enrollments = pd.read_csv('generated/parallel_enrollments.csv')
merge9 = merge8.merge(parallel_enrollments, left_on='enrollment_id', right_on='enrollment_id', how='inner')

week_counts = pd.read_csv('generated/wk_cnts.csv')
merge10 = merge9.merge(week_counts, left_on='enrollment_id', right_on='enrollment_id', how='inner')

encoded_courses = pd.read_csv('generated/encoded_course.csv')
merge11 = merge10.merge(encoded_courses, left_on='enrollment_id', right_on='enrollment_id', how='inner')

daily_activity_count = pd.read_csv('generated/daily_cnts.csv')
merge12 = merge11.merge(daily_activity_count, left_on='enrollment_id', right_on='enrollment_id', how='inner')

week_sessions_count = pd.read_csv('generated/wk_session_cnts.csv')
merge13 = merge12.merge(week_sessions_count, left_on='enrollment_id', right_on='enrollment_id', how='inner')

module_percentage_count = pd.read_csv('generated/percentage_of_module.csv')
merge14 = merge13.merge(module_percentage_count, left_on='enrollment_id', right_on='enrollment_id', how='inner')

merge14.shape

(72395, 132)

In [0]:
merge14.to_csv('merged/finalData.csv', index=False)

In [0]:
  data = pd.read_csv('merged/finalData.csv')
  data.shape

(72395, 132)

Test related features that are derived from Train.

In [0]:
import os
os.chdir('/content/MLProject/submissions/originalData')

In [0]:
!tar -xf test.tar.gz

In [0]:
os.chdir('/content/MLProject/submissions/originalData/test')

In [0]:
!mkdir generated

Generation of course completion rate csv for test from train.

In [0]:
train_course_completion_rate = pd.read_csv('/content/MLProject/submissions/originalData/train/generated/completion_rate_for_courses.csv')
train_enrollment = pd.read_csv('/content/MLProject/submissions/originalData/train/enrollment_train.csv')

train_merged = train_enrollment.merge(train_course_completion_rate, left_on='enrollment_id', right_on='enrollment_id', how='inner')

completion_by_courses = pd.DataFrame({'course_id':train_merged.course_id.unique()})
completion_by_courses['overall_course_completion_rate'] = [list(set(train_merged['overall_course_completion_rate'].loc[train_merged['course_id'] == x['course_id']]))[0]
    for _, x in completion_by_courses.iterrows()]


test_enrollment = pd.read_csv('/content/MLProject/submissions/originalData/test/enrollment_train.csv')
test_enrollment_course_completion = test_enrollment.merge(completion_by_courses, left_on='course_id', right_on='course_id', how='inner')[['enrollment_id', 'overall_course_completion_rate']]
test_enrollment_course_completion.to_csv('/content/MLProject/submissions/originalData/test/generated/completion_rate_for_courses.csv', index=False)

Generation of individual completion rate csv for test from train.

In [0]:
train_individual_completion_rate = pd.read_csv('/content/MLProject/submissions/originalData/train/generated/individual_completion_rate.csv')
train_enrollment = pd.read_csv('/content/MLProject/submissions/originalData/train/enrollment_train.csv')

train_merged = train_enrollment.merge(train_individual_completion_rate, left_on='enrollment_id', right_on='enrollment_id', how='inner')


usernames = pd.DataFrame({'username':train_merged.username.unique()})

usernames['drop_rate'] = [list(set(train_merged['drop_rate'].loc[train_merged['username'] == x['username']]))[0] 
    for _, x in usernames.iterrows()]

usernames['completion_rate'] = [list(set(train_merged['completion_rate'].loc[train_merged['username'] == x['username']]))[0]
    for _, x in usernames.iterrows()]


test_enrollment = pd.read_csv('/content/MLProject/submissions/originalData/test/enrollment_train.csv')

merge_with_completion = test_enrollment.merge(usernames, left_on='username', right_on='username', how='left')

# merge_with_completion = merge_with_completion.fillna(0.5)
# merge_with_completion


merge_with_completion.isnull().sum()
merge_with_completion['completion_rate'].value_counts()

mean_completion_rate = merge_with_completion['completion_rate'].mean(skipna = True) 

mean_drop_rate = merge_with_completion['drop_rate'].mean(skipna = True) 

merge_with_completion[['completion_rate']] = merge_with_completion[['completion_rate']].fillna(mean_completion_rate)
merge_with_completion[['drop_rate']] = merge_with_completion[['drop_rate']].fillna(mean_drop_rate)


final_merge_with_filled_values = merge_with_completion[['enrollment_id', 'drop_rate', 'completion_rate']]

final_merge_with_filled_values.to_csv('/content/MLProject/submissions/originalData/test/generated/individual_completion_rate.csv', index=False)
